In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data=pd.read_csv('./data/features.csv',encoding='utf-8',index_col=0)
data = data
indicateur=data.columns.tolist()   ## nombre des indicateurs
profil=data.index.tolist()    ## nombre des profils
value=data.values

In [4]:
data

,vis,r_fri_follow,avg_fav,frequency
0,0.164286,0.266114,0.925770,0.000306
1,0.578571,0.672932,3.685514,0.015026
2,0.164286,1.037681,0.037716,0.015753
3,0.245714,0.214371,2.944980,0.006128
4,0.164286,0.587054,0.110839,0.009781
...,...,...,...,...
267462,0.264286,0.248337,0.266802,0.001994
267463,0.081429,0.448549,5.806159,0.000793
267464,0.164286,0.552846,0.151943,0.009021
267465,0.330000,0.144140,1.883822,0.015576


In [5]:
### définir la fonction de normalisation, on rajoute une constante 0,01
def std_data(value,flag):
    for i in range(len(indicateur)):
        #print(flag[i])
        if flag[i]=='+':
            value[:,i]=(value[:,i]-np.min(value[:,i],axis=0))/(np.max(value[:,i],axis=0)-np.min(value[:,i],axis=0))+0.01
        elif flag[i]=='-':
            value[:,i]=(np.max(value[:,i],axis=0)-value[:,i])/(np.max(value[:,i],axis=0)-np.min(value[:,i],axis=0))+0.01
    return value

In [6]:
# définir la fonction d'entropie et calculer les poids et les entropies
def cal_weight(indicateur,profil,value):
    p= np.array([[0.0 for i in range(len(indicateur))] for i in range(len(profil))])                    
    #print(p)
    for i in range(len(indicateur)):
        p[:,i]=value[:,i]/np.sum(value[:,i],axis=0)
        
    e=-1/np.log(len(profil))*sum(p*np.log(p))      #calcul d'entropie
    g=1-e     
    w=g/sum(g)     #calcul des poids
    return w

In [7]:
## normalisation des données
flag=["+","+","+","+"]  ## '+' ou '-' désigne le sens positif ou négatif des indicateurs
std_value=std_data(value,flag)
std_value.round(3)

array([[0.086, 0.01 , 0.01 , 0.01 ],
       [0.279, 0.01 , 0.011, 0.016],
       [0.086, 0.01 , 0.01 , 0.016],
       ...,
       [0.086, 0.01 , 0.01 , 0.014],
       [0.163, 0.01 , 0.01 , 0.016],
       [0.087, 0.01 , 0.012, 0.01 ]])

In [8]:
## Résultat
w=cal_weight(indicateur,profil,std_value)

w=pd.DataFrame(w,index=data.columns,columns=['Poids'])
print("#######Poids:#######")
print(w)
score=np.dot(std_value,w).round(2)
score=pd.DataFrame(score,index=data.index,columns=['Points']).sort_values(by =['Points'],ascending = False)

#######Poids:#######
                 Poids
vis           0.528261
r_fri_follow  0.009408
avg_fav       0.030444
frequency     0.431887


In [18]:
score

,Points
44058,0.55
140811,0.54
255943,0.54
135815,0.54
77321,0.54
...,...
168993,0.01
129424,0.01
239404,0.01
255849,0.01


In [14]:
label = score.apply(lambda row: 0 if row["Points"] > 0.06 else 1, axis=1 )

In [15]:
label

44058     0
140811    0
255943    0
135815    0
77321     0
         ..
168993    1
129424    1
239404    1
255849    1
209698    1
Length: 266203, dtype: int64

In [10]:
data["label"] = label

In [17]:
label.to_csv("./data/label.csv")

In [11]:
data.to_csv("./data/feature_labeled.csv")